## Bonus: Eigene Metriken benutzen

Wir haben die drei Standardmetriken der **RAG-Triade** verwendet. Diese sind ein guter Startpunkt, aber für eine praxisorientierte Bewertung reichen sie nicht aus. 

Best-practice ist daher, die eigenen Anforderungen an das RAG-System zu definieren und diese in maßgeschneiderte Metriken zu übersetzen.

Zunächst einmal: Was passiert eigentlich "under-the-hood", wenn wir eine Metrik wie die **Answer Relevancy** von DeepEval nutzen? 

Das Prinzip ist einfach: Wir übergeben die *Frage* und die *generierte Antwort* an ein separates LLM, der die Rolle eines Richters übernimmt. 
Dieser *LLM-as-a-judge* bewertet dann, wie relevant die Antwort auf die Frage ist.

In [22]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

answer_relevance_metric = AnswerRelevancyMetric(model="gpt-4o-mini")
test_case = LLMTestCase(
	input="welche neuen medikamente zur behandlung von psychosen gibt es und wie wirken diese?",
	actual_output="""
	Basierend auf den vorliegenden Informationen gibt es zwei vielversprechende neue Medikamente zur Behandlung von Psychosen:

	1. KarXT (Cobenfy):
	Das Medikament stellt einen bedeutenden Durchbruch in der Behandlung von Schizophrenie dar. Es unterscheidet sich von bisherigen Antipsychotika durch seinen neuartigen Wirkmechanismus. KarXT besteht aus zwei Komponenten: Xanomelin, einem M1/M4 muskarinergen Agonisten, und Trospium, einem peripher beschränkten Anticholinergikum. Der Wirkungsmechanismus zielt auf Muskarinrezeptoren ab und reduziert indirekt die präsynaptische Dopaminausschüttung, im Gegensatz zu herkömmlichen Medikamenten, die postsynaptische Dopaminrezeptoren blockieren.

	2. Emraclidin:
	Dieses Medikament wird als weiterer vielversprechender Ansatz bezeichnet. Es wirkt als positiver allosterischer Modulator am M4-Rezeptor und weist einen ähnlichen Ansatz wie KarXT auf. Die Forschung sieht insbesondere Potenzial für die Behandlung akuter psychotischer Erkrankungen, vor allem bei Positiv-Symptomen und in frühen Erkrankungsphasen. Beide Medikamente eröffnen neue Perspektiven in der Psychosebehandlung, indem sie von der traditionellen Dopamin-Hypothese abweichen.
	""",
)

In [23]:
evaluate([test_case], [answer_relevance_metric])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because the response directly addressed the question about new medications for treating psychosis and their mechanisms of action without any irrelevant statements., error: None)

For test case:

  - input: welche neuen medikamente zur behandlung von psychosen gibt es und wie wirken diese?
  - actual output: 
	Basierend auf den vorliegenden Informationen gibt es zwei vielversprechende neue Medikamente zur Behandlung von Psychosen:

	1. KarXT (Cobenfy):
	Das Medikament stellt einen bedeutenden Durchbruch in der Behandlung von Schizophrenie dar. Es unterscheidet sich von bisherigen Antipsychotika durch seinen neuartigen Wirkmechanismus. KarXT besteht aus zwei Komponenten: Xanomelin, einem M1/M4 muskarinergen Agonisten, und Trospium, einem peripher beschränkten Anticholinergikum. Der Wirkungsmechanismus zielt auf Muskarinrezeptoren ab und reduziert 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the response directly addressed the question about new medications for treating psychosis and their mechanisms of action without any irrelevant statements.', strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.0006444, verbose_logs='Statements:\n[\n    "Es gibt zwei vielversprechende neue Medikamente zur Behandlung von Psychosen.",\n    "KarXT (Cobenfy) stellt einen bedeutenden Durchbruch in der Behandlung von Schizophrenie dar.",\n    "KarXT unterscheidet sich von bisherigen Antipsychotika durch seinen neuartigen Wirkmechanismus.",\n    "KarXT besteht aus zwei Komponenten: Xanomelin und Trospium.",\n    "Xanomelin ist ein M1/M4 muskarinergen Agonisten.",\n    "Trospium ist ein peripher beschränkter Anticholinergikum.",\n    "Der Wirkungsmechanismus von KarXT z

Der Output zeigt uns den **Score** (zwischen 0 und 1) und eine detaillierte **Begründung** für die Entscheidung.

### Beispiel 1: Gesetzestexte zusammenfassen

Doch generische Metriken stoßen schnell an ihre Grenzen, wenn es um spezielle Anwendungsfälle geht. <br>
Wenn unser RAG-System beispielsweise Zusammenfassungen von Gesetzestexten erstellen soll, brauchen wir angepasste Kriterien.

Auch hierfür können wir DeepEval nutzen. Wir definieren einfach in der Variable `evaluation_steps` genau die Kriterien, die für uns entscheidend sind. <br>
Anschließend geben wir erneut die relevanten Variablen an. In unserem Fall: *Gesetzestext* und die *generierte Zusammenfassung*. <br>
Das System läuft dann wie gewohnt und liefert den gleichen Output: einen Score und eine detaillierte Begründung, die sich an unseren selbst definierten Kriterien orientiert.

In [12]:
# Create a mock document class, consisting of the full text and a summary
class Document:
    def __init__(self):
        self.FULL_TEXT = """
        Bürgerliches Gesetzbuch (BGB)
        § 535 Inhalt und Hauptpflichten des Mietvertrags
        (1) Durch den Mietvertrag wird der Vermieter verpflichtet, dem Mieter den Gebrauch der Mietsache während der Mietzeit zu gewähren. 
        Der Vermieter hat die Mietsache dem Mieter in einem zum vertragsgemäßen Gebrauch geeigneten Zustand zu überlassen und sie während der Mietzeit in diesem Zustand zu erhalten. 
        Er hat die auf der Mietsache ruhenden Lasten zu tragen.
        (2) Der Mieter ist verpflichtet, dem Vermieter die vereinbarte Miete zu entrichten.
        ...
        """

        self.SUMMARY = """
        Ein Mietvertrag verpflichtet den Vermieter, die Immobilie in einem nutzbaren Zustand bereitzustellen und zu erhalten, während der Mieter dafür die vereinbarte Miete zahlen muss.
        """

# Create the document instance
document = Document()

In [25]:
from deepeval import evaluate
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

summarization_correctness_metric = GEval(
	name="Summarization_Correctness",
	evaluation_steps=[
		"Prüfe, ob die wesentlichen rechtlichen Inhalte korrekt wiedergegeben werden.",
		"Überprüfe, ob keine zusätzlichen Informationen erfunden wurden.",
		"Stelle sicher, dass die Zusammenfassung den Sinn und die rechtliche Bedeutung nicht verfälscht.",
		"Achte darauf, dass die Sprache klar und für juristische Laien nachvollziehbar bleibt."
	],
	evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.INPUT],
)

llm_test_case_summary = LLMTestCase(
	input=document.FULL_TEXT,
	actual_output=document.SUMMARY,
)

In [26]:
evaluate([llm_test_case_summary], [summarization_correctness_metric])

✨ You're running DeepEval's latest Summarization_Correctness [GEval] Metric! (using gpt-4.1, strict=False, 
async_mode=True)...

/usr/local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Metrics Summary

  - ✅ Summarization_Correctness [GEval] (score: 0.8010986942630594, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: Die wesentlichen rechtlichen Inhalte des § 535 BGB werden korrekt wiedergegeben: Die Pflichten des Vermieters (Bereitstellung und Erhaltung der Mietsache) und des Mieters (Zahlung der Miete) sind enthalten. Es wurden keine zusätzlichen Informationen erfunden. Allerdings fehlt der Hinweis auf die Pflicht des Vermieters, die auf der Mietsache ruhenden Lasten zu tragen. Die Zusammenfassung ist klar und für Laien verständlich, verfälscht den Sinn nicht, lässt aber einen Aspekt des Gesetzestextes aus., error: None)

For test case:

  - input: 
        Bürgerliches Gesetzbuch (BGB)
        § 535 Inhalt und Hauptpflichten des Mietvertrags
        (1) Durch den Mietvertrag wird der Vermieter verpflichtet, dem Mieter den Gebrauch der Mietsache während der Mietzeit zu gewähren. 
        Der Vermieter hat die Mietsache dem Mieter in einem zum ver

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Summarization_Correctness [GEval]', threshold=0.5, success=True, score=0.8010986942630594, reason='Die wesentlichen rechtlichen Inhalte des § 535 BGB werden korrekt wiedergegeben: Die Pflichten des Vermieters (Bereitstellung und Erhaltung der Mietsache) und des Mieters (Zahlung der Miete) sind enthalten. Es wurden keine zusätzlichen Informationen erfunden. Allerdings fehlt der Hinweis auf die Pflicht des Vermieters, die auf der Mietsache ruhenden Lasten zu tragen. Die Zusammenfassung ist klar und für Laien verständlich, verfälscht den Sinn nicht, lässt aber einen Aspekt des Gesetzestextes aus.', strict_mode=False, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.00199, verbose_logs='Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Prüfe, ob die wesentlichen rechtlichen Inhalte korrekt wiedergegeben werden.",\n    "Überprüfe, ob keine zusätzlichen Informationen erfunden wur

### Beispiel 2: RAG QA-Bot für Schüler:innen

Für einen RAG-Bot, der Antworten für Schüler:innen generiert, sind andere Kriterien entscheidend. Wir definieren erneut die Evaluierungsschritte, um sicherzustellen, dass die Antworten einfach und altersgerecht sind.  

Im Unterschied zum vorherigen Beispiel wollen wir hier keinen kontinuierlichen Score (zwischen 0 und 1), sondern eine klare Aussage: 'pass' oder 'fail'. 

Dafür setzen wir einfach die Variable `strict_mode` auf `True`. Der restliche Prozess bleibt derselbe.

In [27]:
student_comprehensibility = GEval(
	name="Student_Comprehensibility",
    evaluation_steps=[
        "Bewerte, ob einfache, altersgerechte Sprache verwendet wird (kurze Sätze, aktive Form).",
        "Kennzeichne Fachbegriffe/Jargon und pruefe, ob sie vermieden oder kurz erklärt werden.",
        "Achte auf klare Struktur (1–3 Kernpunkte, ggf. kurzes Beispiel).",
        "Überprüfe, ob keine überfluessigen Details die Verständlichkeit mindern."
    ],
	evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.INPUT],
	strict_mode=True, # strict mode -> binärer output (0 oder 1)
)

INPUT_QUESTION = "Was genau bedeutet Klimawandel?"

OUTPUT_ANSWER = (
    "Der Klimawandel bezeichnet signifikante statistische Verschiebungen in der "
    "Energie- und Strahlungsbilanz des Klimasystems, induziert durch anthropogene "
    "Forcings und nichtlineare Rückkopplungsmechanismen."
)

llm_test_case_school = LLMTestCase(
	input=INPUT_QUESTION,
	actual_output=OUTPUT_ANSWER,
)

In [28]:
evaluate([llm_test_case_school], [student_comprehensibility])
# Hier erwarten wir einen **fail**-Output, da die Antwort zu komplex ist. 

✨ You're running DeepEval's latest Student_Comprehensibility [GEval] Metric! (using gpt-4.1, strict=True, 
async_mode=True)...

/usr/local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Metrics Summary

  - ❌ Student_Comprehensibility [GEval] (score: 0.0, threshold: 1.0, strict: True, evaluation model: gpt-4.1, reason: The output uses complex terms like 'statistische Verschiebungen', 'Energie- und Strahlungsbilanz', 'anthropogene Forcings', and 'nichtlineare Rückkopplungsmechanismen' without explanation, does not use simple language, and lacks a clear, concise structure suitable for the input question., error: None)

For test case:

  - input: Was genau bedeutet Klimawandel?
  - actual output: Der Klimawandel bezeichnet signifikante statistische Verschiebungen in der Energie- und Strahlungsbilanz des Klimasystems, induziert durch anthropogene Forcings und nichtlineare Rückkopplungsmechanismen.
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Student_Comprehensibility [GEval]: 0.00% pass rate




✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Student_Comprehensibility [GEval]', threshold=1.0, success=False, score=0.0, reason="The output uses complex terms like 'statistische Verschiebungen', 'Energie- und Strahlungsbilanz', 'anthropogene Forcings', and 'nichtlineare Rückkopplungsmechanismen' without explanation, does not use simple language, and lacks a clear, concise structure suitable for the input question.", strict_mode=True, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.001298, verbose_logs='Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Bewerte, ob einfache, altersgerechte Sprache verwendet wird (kurze Sätze, aktive Form).",\n    "Kennzeichne Fachbegriffe/Jargon und pruefe, ob sie vermieden oder kurz erklärt werden.",\n    "Achte auf klare Struktur (1–3 Kernpunkte, ggf. kurzes Beispiel).",\n    "Überprüfe, ob keine überfluessigen Details die Verständlichkeit mindern."\n] \n \nRubric:\nNone \n \nScore

In [29]:
INPUT_QUESTION = "Was genau bedeutet Klimawandel?"

OUTPUT_ANSWER = (
    "Klimawandel heißt, dass es auf der Erde nach und nach wärmer wird, "
    "weil wir Menschen viele Abgase in die Luft pusten. Dadurch ändert sich das Wetter."
)

llm_test_case_school = LLMTestCase(
    input=INPUT_QUESTION,
    actual_output=OUTPUT_ANSWER,
)

In [30]:
evaluate([llm_test_case_school], [student_comprehensibility])
# Hier erwarten wir einen **pass**-Output, da die Antwort nun altersgerecht ist. 

✨ You're running DeepEval's latest Student_Comprehensibility [GEval] Metric! (using gpt-4.1, strict=True, 
async_mode=True)...

/usr/local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Metrics Summary

  - ✅ Student_Comprehensibility [GEval] (score: 1.0, threshold: 1.0, strict: True, evaluation model: gpt-4.1, reason: Die Antwort verwendet einfache, altersgerechte Sprache mit kurzen Sätzen und aktiver Form, erklärt den Fachbegriff 'Klimawandel' verständlich, enthält 1–2 Kernpunkte und vermeidet überflüssige Details., error: None)

For test case:

  - input: Was genau bedeutet Klimawandel?
  - actual output: Klimawandel heißt, dass es auf der Erde nach und nach wärmer wird, weil wir Menschen viele Abgase in die Luft pusten. Dadurch ändert sich das Wetter.
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Student_Comprehensibility [GEval]: 100.00% pass rate




✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Student_Comprehensibility [GEval]', threshold=1.0, success=True, score=1.0, reason="Die Antwort verwendet einfache, altersgerechte Sprache mit kurzen Sätzen und aktiver Form, erklärt den Fachbegriff 'Klimawandel' verständlich, enthält 1–2 Kernpunkte und vermeidet überflüssige Details.", strict_mode=True, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.0011359999999999999, verbose_logs='Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Bewerte, ob einfache, altersgerechte Sprache verwendet wird (kurze Sätze, aktive Form).",\n    "Kennzeichne Fachbegriffe/Jargon und pruefe, ob sie vermieden oder kurz erklärt werden.",\n    "Achte auf klare Struktur (1–3 Kernpunkte, ggf. kurzes Beispiel).",\n    "Überprüfe, ob keine überfluessigen Details die Verständlichkeit mindern."\n] \n \nRubric:\nNone \n \nScore: 1')], conversational=False, multimodal=False, input='Was genau bedeutet Kl

### Beispiel 3: RAG QA-Bot mir kurzen und prägnanten Antworten

Für manche Anwendungsfälle brauchen wir keine komplexen LLM-Bewertungen. 
Einfache, **deterministische Metriken** sind oft schneller, effizienter und günstiger. 
Ein Beispiel ist die Überprüfung der Antwortlänge: 
Wir können eine simple Funktion schreiben, die zählt, ob die Antwort eine definierte Maximallänge (z.B. 500 Zeichen) nicht überschreitet, und uns dann ein klares 'Ja' oder 'Nein' zurückgibt.

In [21]:
def length_metric(test_case: LLMTestCase, max_len=500):
    return 1 if len(test_case.actual_output) <= max_len else 0

INPUT_QUESTION = "Was genau bedeutet Klimawandel?"

OUTPUT_ANSWER = (
    "Klimawandel heißt, dass es auf der Erde nach und nach wärmer wird, "
    "weil wir Menschen viele Abgase in die Luft pusten. Dadurch ändert sich das Wetter."
)

llm_test_case = LLMTestCase(
    input=INPUT_QUESTION,
    actual_output=OUTPUT_ANSWER,
)

print(length_metric(llm_test_case))

1
